In [8]:
import pandas as pd
import numpy as np

In [32]:
houses = pd.read_csv("https://raw.githubusercontent.com/Johnle3/MLHousingPrices/main/redfin_2022-06-06-11-05-25.csv")
# Remove unnecessary columns
houses = houses.drop('FAVORITE', axis=1)
houses = houses.drop(['SALE TYPE', 'SOLD DATE','ADDRESS','HOA/MONTH', 'NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','INTERESTED','URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)','SOURCE', 'MLS#','STATUS','LOCATION','DAYS ON MARKET'], axis = 1)

#Subset to only Alexandria and without Vacant Lot
houses = houses[houses['CITY']!= 'Arlington']
houses = houses[houses['PROPERTY TYPE'] != "Vacant Land"]
houses = houses[houses['PROPERTY TYPE'] != "Condo/Co-op"]
# Drop null values
#houses = houses.dropna()
houses = houses.dropna(subset=['LOT SIZE', 'YEAR BUILT'], how='any')
houses = houses[houses['PROPERTY TYPE'] != "Multi-Family (2-4 Unit)"]
houses.head()

,PROPERTY TYPE,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,LATITUDE,LONGITUDE
0,Single Family Residential,Alexandria,VA,22308,45000000,7.0,10.0,25500.0,435600.0,2023.0,1765.0,38.739577,-77.045973
1,Single Family Residential,Alexandria,VA,22314,5400000,6.0,5.0,8145.0,21153.0,1795.0,663.0,38.809834,-77.044992
4,Single Family Residential,Alexandria,VA,22309,3650000,4.0,5.0,5602.0,21972.0,2010.0,652.0,38.699743,-77.116651
7,Single Family Residential,Alexandria,VA,22309,3250000,6.0,6.5,5590.0,105654.0,2021.0,581.0,38.703304,-77.094285
8,Single Family Residential,Alexandria,VA,22314,3195000,4.0,3.5,4122.0,3116.0,1800.0,775.0,38.803387,-77.044415


In [33]:
# null values
# houses.isna().sum()

In [34]:
# create buckets for year built
i = len(houses.index)
j = 0
while j < i:
    year = houses.iloc[j, 9]
    
    if 1700 <= year < 1900:
        houses.iloc[j, 9] = "18th-19th Century"
    elif 1900 <= year < 1950:
        houses.iloc[j, 9] = '1900-1950'
    elif 1950 <= year < 2000:
        houses.iloc[j, 9] = '1950-2000'
    elif 2000 <= year <= 2023:
        houses.iloc[j, 9] = '2000-2023'
    else:
        pass
    j = j + 1

In [35]:
from sklearn.model_selection import train_test_split
X,y= houses[['PROPERTY TYPE','BEDS','BATHS','SQUARE FEET', 'LOT SIZE','YEAR BUILT']], houses[['PRICE']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=18)
X_columns = X_train.columns
X_train.head(3)

,PROPERTY TYPE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT
126,Townhouse,3.0,4.5,1928.0,1752.0,18th-19th Century
82,Single Family Residential,5.0,2.5,2942.0,6930.0,1900-1950
267,Single Family Residential,3.0,1.5,1750.0,3700.0,1950-2000


In [36]:
#one-hot encode 
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
column_trans = make_column_transformer((OneHotEncoder(), [0,5]), remainder='passthrough')

In [53]:
X_train = column_trans.fit_transform(X_train)
X_test = column_trans.transform(X_test)
X_train[:1]

array([[0.000e+00, 1.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        3.000e+00, 4.500e+00, 1.928e+03, 1.752e+03]])

In [54]:
#scaling 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test[:1]

array([[ 0.68175416, -0.68175416, -0.27878344, -0.52542708,  1.04929028,
        -0.55513611, -0.90652299, -0.83857917, -0.91493715, -0.52372109]])

In [56]:
# Lasso Regression Model

from sklearn import linear_model
clf = linear_model.Lasso(alpha =0.1)
clf.fit(X_train, y_train)

print(clf.coef_)

[-4.53010474e+04  1.69249964e-10  1.81310457e+05  8.60076972e+04
 -1.94353222e+04  4.77151759e+03 -1.47392533e+03  6.17751912e+04
  2.72473713e+05  5.56777181e+04]


/Users/johnle/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1204397591468.996, tolerance: 5808873719.196496
  model = cd_fast.enet_coordinate_descent(


In [58]:
print(clf.intercept_)

[1242279.86057692]


# Different Way of Lasso Regression

In [59]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
model = Lasso(alpha=1.0)

In [60]:
from numpy import arange
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV


In [61]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [62]:
X,y= houses[['PROPERTY TYPE','BEDS','BATHS','SQUARE FEET', 'LOT SIZE','YEAR BUILT']], houses[['PRICE']]
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE: nan (nan)


In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=18)
X_columns = X_train.columns

In [47]:
grid = dict()
grid['alpha'] = arange(0, 1, 0.01)

In [48]:
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [64]:
from pandas import read_csv
from sklearn.linear_model import Lasso
# load the dataset
# define model
model = Lasso(alpha=1.0)
# fit model
model.fit(X, y)
# define new data
row = [2,2,10000,2000,2000]
# make a prediction
yhat = model.predict([row])
# summarize prediction
print('Predicted: %.3f' % yhat)

ValueError: could not convert string to float: 'Single Family Residential'

In [65]:
row = [2,2,10000,2000,2020]
# make a prediction
yhat = model.predict([row])
print('Predicted: %.3f' % yhat)

NotFittedError: This Lasso instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.